# Motivation

Next, I want to run some topic modeling algorithms using the chapters of the Bible as my corpus. I'm curious if the chapters in the Bible will cluster together in some logical way, i.e. New Testament chapters, Psalms chapters, Minor Prophet chapters, etc. If not, will there be some other discernable pattern?

In order to do this I will first collapse all of the text in each chapter into a single observation. I will then process that data and add the tokenized and clean list of words from each chapter to a corpus list. I will then apply various topic modeling algorithms to this corpus and analyze the results for the best fit. Once that's done, I will determine which chapters belong to the various topics. From here, I can start to answer the questions I have about topic modeling the Bible.

I will use the gensim package for my analysis. This package offers several transformation methods that we will explore.

# Set up

This is my typical set up. I import the packages I will use, set my project directory, remove column and row limits, and allow Jupyter to display all of the output from each cell.

In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3
import spacy
from datetime import datetime

# Set project folder as directory
os.chdir(r'C:/Users/david/Projects/Bible Analytics')

# Remove row and column limits
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Display all output from each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Accessing data

In [2]:
database = 'Data/SQL database.db'

In [3]:
conn = sqlite3.connect(database)
 
df = pd.read_sql_query('SELECT * FROM t_web', conn)
 
conn.close

<function Connection.close()>

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31102 entries, 0 to 31101
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     31102 non-null  object
 1   old_new  31102 non-null  object
 2   group    31102 non-null  int64 
 3   id       31102 non-null  int64 
 4   b        31102 non-null  int64 
 5   c        31102 non-null  int64 
 6   v        31102 non-null  int64 
 7   t        31102 non-null  object
 8   clean_t  31102 non-null  object
dtypes: int64(5), object(4)
memory usage: 2.1+ MB


,name,old_new,group,id,b,c,v,t,clean_t
0,Genesis,OT,1,1001001,1,1,1,"In the beginning God{After ""God,"" the Hebrew has the two letters ""Aleph Tav"" (the first and last letters of the Hebrew alphabet) as a grammatical marker.} created the heavens and the earth.",In the beginning God created the heavens and the earth.
1,Genesis,OT,1,1001002,1,1,2,Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.,Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.
2,Genesis,OT,1,1001003,1,1,3,"God said, ""Let there be light,"" and there was light.","God said, ""Let there be light,"" and there was light."
3,Genesis,OT,1,1001004,1,1,4,"God saw the light, and saw that it was good. God divided the light from the darkness.","God saw the light, and saw that it was good. God divided the light from the darkness."
4,Genesis,OT,1,1001005,1,1,5,"God called the light Day, and the darkness he called Night. There was evening and there was morning, one day.","God called the light Day, and the darkness he called Night. There was evening and there was morning, one day."


# Begin

The first thing I will do is combine all of the text in each chapter into a single observation. This is actually quite simple. I will group our data by book and chapter and apply the sum function to our clean text column. Then I will reset the index so that book and chapter show up as columns. Finally, I will merge this data with out key dataset that contains the books names. Finally, I will reorder the columns to suit my personal preferences.

In [5]:
chapter_text = pd.DataFrame(df.groupby(['b','c'])['clean_t'].sum())
chapter_text.reset_index(inplace=True)

key = pd.read_csv('Jupyter/Jupyter data/key_english.csv')

chapter_text = key.merge(chapter_text, how='inner', left_on='b', right_on='b')
chapter_text = chapter_text[['name', 'old_new', 'group', 'b', 'c', 'clean_t']]

chapter_text.head(2)

,name,old_new,group,b,c,clean_t
0,Genesis,OT,1,1,1,"In the beginning God created the heavens and the earth.Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.God said, ""Let there be light,"" and there was light.God saw the light, and saw that it was good. God divided the light from the darkness.God called the light Day, and the darkness he called Night. There was evening and there was morning, one day.God said, ""Let there be an expanse in the midst of the waters, and let it divide the waters from the waters.""God made the expanse, and divided the waters which were under the expanse from the waters which were above the expanse, and it was so.God called the expanse sky. There was evening and there was morning, a second day.God said, ""Let the waters under the sky be gathered together to one place, and let the dry land appear,"" and it was so.God called the dry land Earth, and the gathering together of the waters he called Seas. God saw that it was good.God said, ""Let the earth put forth grass, herbs yielding seed, and fruit trees bearing fruit after their kind, with its seed in it, on the earth,"" and it was so.The earth brought forth grass, herbs yielding seed after their kind, and trees bearing fruit, with its seed in it, after their kind: and God saw that it was good.There was evening and there was morning, a third day.God said, ""Let there be lights in the expanse of sky to divide the day from the night; and let them be for signs, and for seasons, and for days and years;and let them be for lights in the expanse of sky to give light on the earth,"" and it was so.God made the two great lights: the greater light to rule the day, and the lesser light to rule the night. He also made the stars.God set them in the expanse of sky to give light to the earth,and to rule over the day and over the night, and to divide the light from the darkness. God saw that it was good.There was evening and there was morning, a fourth day.God said, ""Let the waters swarm with swarms of living creatures, and let birds fly above the earth in the open expanse of sky.""God created the large sea creatures, and every living creature that moves, with which the waters swarmed, after their kind, and every winged bird after its kind. God saw that it was good.God blessed them, saying, ""Be fruitful, and multiply, and fill the waters in the seas, and let birds multiply on the earth.""There was evening and there was morning, a fifth day.God said, ""Let the earth bring forth living creatures after their kind, cattle, creeping things, and animals of the earth after their kind,"" and it was so.God made the animals of the earth after their kind, and the cattle after their kind, and everything that creeps on the ground after its kind. God saw that it was good.God said, ""Let us make man in our image, after our likeness: and let them have dominion over the fish of the sea, and over the birds of the sky, and over the cattle, and over all the earth, and over every creeping thing that creeps on the earth.""God created man in his own image. In God's image he created him; male and female he created them.God blessed them. God said to them, ""Be fruitful, multiply, fill the earth, and subdue it. Have dominion over the fish of the sea, over the birds of the sky, and over every living thing that moves on the earth.""God said, ""Behold, I have given you every herb yielding seed, which is on the surface of all the earth, and every tree, which bears fruit yielding seed. It will be your food.To every animal of the earth, and to every bird of the sky, and to everything that creeps on the earth, in which there is life, I have given every green herb for food."" And it was so.God saw everything that he had made, and, behold, it was very good. There was evening and there was morning, the sixth day."
1,Genesis,OT,1,1,2,"The heavens and the earth were finished, and all the host of them.On the seventh da

# Pushing chapter data to SQL database

Before moving forward, I want to save this dataset to our SQL dataset.

In [6]:
conn = sqlite3.connect(database)

chapter_text.to_sql('chapter_text', conn, if_exists='replace', index=False)

conn.close()

1189

In [7]:
# *table* means double quotes around table
 
conn = sqlite3.connect(database)
cursor = conn.cursor()
 
cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
 
for i in cursor.fetchall():
    print(i[0])
    
conn.close()

t_web
people_names
gpe_name
chapter_text


# Compiling stop words from NLP packages

Next, I will clean the data by removing what are called "stop words." These are commonly used words that would tell us very little about any particular topic, e.g. he, they, are, and. There are several NLP packages that have their own lists of stop words. I've examined each of these lists in the past and I'm comfortable removing the stop words contained in each list. As such, I will combine these list into a single list that I will then use for processing. I will also add some additional words that are specific to the Bible. 

In [8]:
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

from gensim.parsing.preprocessing import STOPWORDS
gen_stopwords = list(STOPWORDS)

nlp = spacy.load('en_core_web_lg')
spacy_stopwords = list(nlp.Defaults.stop_words)

print('There are', len(nltk_stopwords), 'stopwords in nltk,', len(gen_stopwords), 'in gensim, and', len(spacy_stopwords), 'in spacy.')

There are 179 stopwords in nltk, 337 in gensim, and 326 in spacy.


In [9]:
stopwords = list(set(nltk_stopwords+gen_stopwords+spacy_stopwords+['shall', 'let', 'come', 'go', 'know', 'like']))
len(stopwords)

sorted(stopwords)

417

["'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'ain',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'aren',
 "aren't",
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'come',
 'computer',
 'con',
 'could',
 'couldn',
 "couldn't",
 'couldnt',
 'cry',
 'd',
 'de',
 'describe',
 'detail',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven

# Processing the text

I will also lemmatize the data after removing stopwords. Lemmatizing the text will revert each word back to its root. For instance, "run" will be changed to "ran." By doing this, we ensure run and ran are treated as the same word.

I will begin this by defining an nlp object and loading the large, English language pipeline. I will then define an empty corpus list that we will use for topic modeling. There's some stuff about timing. Then, within a FOR loop I will iterate through the text list for each chapter. First, I'll define an empty list called temp. Then, I will create a TRY block to handle any exceptions. Within this TRY block, I will create a document for each chapter by applying our nlp object to the chapter text. Then, I will created a nested FOR loop and iterate through each word of each chapter. Within this nested FOR loop, I first filter out any proper nouns. I've thought about this and decided I didn't want the topics to be driven by the people and places mentioned in each chapter. By removing proper nouns, I hope these topics will be more content driven. Next, I will use conditional statements to remove any word whose lemmatized version is in our customized list of stop words. For instance, "say" is in our list of stop words, so "said" would also be removed. I also removed punctuation. Finally each lemmatized token is added to the list of words for that chapter. Once every token in a chapter has been iterated through, the entire list of lammetized words is added to our corpus list. The only thing left to mention is that an EXCEPT block is included to handle any errors and tell us where those errors occur should we need to investigate.

In [10]:
chapter_text_sub = chapter_text.head(1)

In [11]:
nlp = spacy.load("en_core_web_lg")

corpus_list = []

# Ignore this
start = datetime.now()
# Stop ignoring

for index, row in chapter_text.iterrows():
    
    temp=[]
    
    try:
        
        doc = nlp(row['clean_t'])        
    
        for token in doc:
            
            if token.pos_ != 'PROPN':
                
                # Removing stopwords
                if token.lemma_.lower() not in stopwords:
                    if not token.is_punct:
                        
                        temp+=[token.lemma_.lower()]
                    
        corpus_list.append(temp)
                
    except:
        
        print('Check out this chapter:')
        print(row['name'], row['c'])        
        print()

# Ignore this
stop = datetime.now()

print('This process took', stop-start)
print()


This process took 0:01:58.555867



In [12]:
corpus_list[0]

['beginning',
 'create',
 'heaven',
 'earth',
 'earth',
 'formless',
 'darkness',
 'surface',
 'deep',
 'hover',
 'surface',
 'water',
 'light',
 'light',
 'light',
 'good',
 'divide',
 'light',
 'darkness',
 'light',
 'darkness',
 'evening',
 'morning',
 'day',
 'expanse',
 'midst',
 'water',
 'divide',
 'water',
 'water',
 'expanse',
 'divide',
 'water',
 'expanse',
 'water',
 'expanse',
 'expanse',
 'sky',
 'evening',
 'morning',
 'second',
 'day',
 'water',
 'sky',
 'gather',
 'place',
 'dry',
 'land',
 'appear',
 'dry',
 'land',
 'gathering',
 'water',
 'good',
 'earth',
 'forth',
 'grass',
 'herb',
 'yield',
 'seed',
 'fruit',
 'tree',
 'bear',
 'fruit',
 'kind',
 'seed',
 'earth',
 'earth',
 'bring',
 'forth',
 'grass',
 'herb',
 'yield',
 'seed',
 'kind',
 'tree',
 'bear',
 'fruit',
 'seed',
 'kind',
 'good',
 'evening',
 'morning',
 'day',
 'light',
 'expanse',
 'sky',
 'divide',
 'day',
 'night',
 'sign',
 'season',
 'day',
 'years;and',
 'light',
 'expanse',
 'sky',
 'light'

This code took just over two minutes and looks good.

# Converting corpus_list into a corpus

Before I can get started I need to convert my list of lists into an actual corpus.

In [13]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(corpus_list)

# Term Document Frequency
chapter_corpus = [id2word.doc2bow(text) for text in corpus_list]

# Topic modeling

The Bible covers a lot of subjects so I will set the number of topics to 100. We'll evaluate this later and either increase or decrease it. Then I'm going to run our LDA on the chapter_corpus

In [14]:
import gensim
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=chapter_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [16]:
import pyLDAvis
import pyLDAvis.gensim_models

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, chapter_corpus, id2word)

LDAvis_prepared

C:\Users\david\anaconda3\envs\linked\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.006578 -0.000146       1        1  26.833225
2     -0.004997 -0.010607       2        1  24.780393
1     -0.005466  0.010795       3        1  17.659890
4      0.011458  0.000901       4        1  16.318290
3      0.005583 -0.000944       5        1  14.408202, topic_info=                   Term         Freq        Total Category  logprob  loglift
247                 son  3005.000000  3005.000000  Default  30.0000  30.0000
94               father  1544.000000  1544.000000  Default  29.0000  29.0000
443               house  2077.000000  2077.000000  Default  28.0000  28.0000
147               child  1951.000000  1951.000000  Default  27.0000  27.0000
14                  day  2444.000000  2444.000000  Default  26.0000  26.0000
72                thing  1410.000000  1410.000000  Default  25.0000  25.0000
487                king  2585.000000  2585.000000  Default  24.0000  24.0000
91                  eat   794.000000   794.000000  Default  23.0000  23.0000
186                tell   900.000000   900.000000  Default  22.0000  22.0000
44                 land  1712.000000  1712.000000  Default  21.0000  21.0000
394             servant   976.000000   976.000000  Default  20.0000  20.0000
206             brother   886.000000   886.000000  Default  19.0000  19.0000
179                send   898.000000   898.000000  Default  18.0000  18.0000
52                  man  3685.000000  3685.000000  Default  17.0000  17.0000
167                hand  1768.000000  1768.000000  Default  16.0000  16.0000
288               heart   895.000000   895.000000  Default  15.0000  15.0000
93                 fall   563.000000   563.000000  Default  14.0000  14.0000
87                  die   540.000000   540.000000  Default  13.0000  13.0000
513                away   944.000000   944.000000  Default  12.0000  12.0000
136               woman   520.000000   520.000000  Default  11.0000  11.0000
449              praise   262.000000   262.000000  Default  10.0000  10.0000
643                love   644.000000   644.000000  Default   9.0000   9.0000
267                year   853.000000   853.000000  Default   8.0000   8.0000
702               death   365.000000   365.000000  Default   7.0000   7.0000
168                hear  1126.000000  1126.000000  Default   6.0000   6.0000
46                 life   480.000000   480.000000  Default   5.0000   5.0000
60                  sea   356.000000   356.000000  Default   4.0000   4.0000
272                 ark   210.000000   210.000000  Default   3.0000   3.0000
137                work   761.000000   761.000000  Default   2.0000   2.0000
222              happen   645.000000   645.000000  Default   1.0000   1.0000
1656             louder     1.984074     3.009184   Topic1 -10.2767   0.8990
3546           mulberry     3.196063     5.002553   Topic1  -9.8000   0.8675
5002            "yahweh     1.244783     1.991718   Topic1 -10.7429   0.8455
5003        seas;though     1.244484     1.991463   Topic1 -10.7432   0.8454
2305              aroma     2.958643     4.951539   Topic1  -9.8772   0.8006
3544           marching     1.762985     2.982535   Topic1 -10.3949   0.7898
8745           us);that     1.160404     1.984938   Topic1 -10.8131   0.7787
4609           outskirt     1.156253     1.980154   Topic1 -10.8167   0.7775
8457          reception     1.159312     1.986222   Topic1 -10.8141   0.7771
2174           swelling     1.740314     2.983440   Topic1 -10.4078   0.7765
5844             seraph     1.143455     1.977898   Topic1 -10.8278   0.7675
8744           life(and     1.144565     1.983607   Topic1 -10.8269   0.7656
5841              'make     1.140270     1.977511   Topic1 -10.8306   0.7650
4301            banquet     7.874821    13.755288   Topic1  -8.8982   0.7578
6558        portion].by     7.162589    12.538335   Topic1  -8.9930   0.7556
5842         glory!"the     1.126700 